Student: Duncan Ferguson <br>
Student Id: 871641260 <br>
Class: Comp 4431-1 <br>
Assignment: Assignment 3 <br>
Date: 11/20/2021 <br>

You are to apply the following techniques to mine this data set:
- Association Rules
- Decision tree Classification
- Naive Bayes Classification
- kmeans and dbscan clustering

Also, you are to use Principle Component Analysis to reduce the number of attributes
(aka features)

Please answer the following questions:

1. What can you deduce from the data set? (in otherwords, what attribute values are indicative of "success")
2. Which mining techniques yielded the best results for what? How do you define "best results"? Please
include tables and/or graphs to justify your statement

Note: While numeric data is desired for classification/clustering, you need strings that look like market basket
data (sets of item names) for Association Rules. Numeric data can be grouped into names like
"lowValue, midValue, midValue, mediumHighValue, highValue" etc. See the following code for possible
way of doing this (not necessarily the best):

What to turn in:
- a pdf file with the answers to these three questions
- Your code used to answer these questions


In [127]:
# Importing Standard Libraries
import pandas as pd
import numpy as np

# Sklearn Libraries
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.naive_bayes import *
from sklearn.cluster import KMeans, DBSCAN
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# Graphing Libraries
import matplotlib.pyplot as plt
import seaborn as sns

In [128]:
# Setting up view options
pd.set_option('display.max_columns', None)
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
pd.options.display.width = 0   #


In [129]:
# Importing the Data Set
df =  pd.read_csv("bank.csv")
# df =  pd.read_csv("bank-full.csv")
df.head()

# Establishing copies for running through the different scrubs
df_a = df.copy()
df_b = df.copy()

### Association Rules Data Mining
First Cleaning up the data and creating categories

In [130]:
for index, row in df_a.iterrows():
	if row['balance'] < 0:
		row['balance'] = 'inDebt'
	else:
		row['balance'] = 'notInDebt'

# Creating Categories for balance
df_a.loc[(df_a['balance'] >= 10000), 'balanceSummary'] = 'veryPositive'
df_a.loc[((df_a['balance'] < 0) & (df_a['balance'] >= -10000)), 'balanceSummary'] = 'negative'
df_a.loc[((df_a['balance'] < 10000) & (df_a['balance'] >= 0)), 'balanceSummary'] = 'positive'
df_a.loc[df_a['balance'] < -500, 'balanceSummary'] = 'veryNegative'

# Creating Categories for ages
df_a.loc[((df_a['age'] < 25) & (df_a['age'] >= 0)),  'ageBand'] = 'ageBand1'
df_a.loc[((df_a['age'] < 30) & (df_a['age'] >= 25)), 'ageBand'] = 'ageBand2'
df_a.loc[((df_a['age'] < 40) & (df_a['age'] >= 30)), 'ageBand'] = 'ageBand3'
df_a.loc[((df_a['age'] < 50) & (df_a['age'] >= 40)), 'ageBand'] = 'ageBand4'
df_a.loc[((df_a['age'] < 120) & (df_a['age'] >= 50)),'ageBand'] = 'ageBand5'

# Creating Categories for default
df_a.loc[(df_a['default'] == 'no'), 'defaultValue'] = 'defaultNo'
df_a.loc[(df_a['default'] == 'yes'), 'defaultValue'] = 'defaultYes'

# Creating Categories for Housing
df_a.loc[(df_a['housing'] == 'no'), 'housingVal'] = 'houseNo'
df_a.loc[(df_a['housing'] == 'yes'), 'housingVal'] = 'houseYes'

# Creating Categories for Loans
df_a.loc[(df_a['loan'] == 'no'), 'loanVal'] = 'loanNo'
df_a.loc[(df_a['loan'] == 'yes'), 'loanVal'] = 'loanYes'

Displaying Data with new categories

In [131]:
df_a.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,balanceSummary,ageBand,defaultValue,housingVal,loanVal
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no,positive,ageBand3,defaultNo,houseNo,loanNo
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no,positive,ageBand3,defaultNo,houseYes,loanYes
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no,positive,ageBand3,defaultNo,houseYes,loanNo
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no,positive,ageBand3,defaultNo,houseYes,loanYes
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no,positive,ageBand5,defaultNo,houseYes,loanNo
5,35,management,single,tertiary,no,747,no,no,cellular,23,feb,141,2,176,3,failure,no,positive,ageBand3,defaultNo,houseNo,loanNo
6,36,self-employed,married,tertiary,no,307,yes,no,cellular,14,may,341,1,330,2,other,no,positive,ageBand3,defaultNo,houseYes,loanNo
7,39,technician,married,secondary,no,147,yes,no,cellular,6,may,151,2,-1,0,unknown,no,positive,ageBand3,defaultNo,houseYes,loanNo
8,41,entrepreneur,married,tertiary,no,221,yes,no,unknown,14,may,57,2,-1,0,unknown,no,positive,ageBand4,defaultNo,houseYes,loanNo
9,43,services,married,primary,no,-88,yes,yes,cellular,17,apr,313,1,147,2,failure,no,negative,ageBand4,defaultNo,houseYes,loanYes


Dropping data that is not in a string

In [132]:
df_a = df_a.drop('age',axis=1)
df_a = df_a.drop('day',axis=1)
df_a = df_a.drop('duration',axis=1)
df_a = df_a.drop('balance',axis=1)
df_a = df_a.drop('campaign',axis=1)
df_a = df_a.drop('previous',axis=1)
df_a = df_a.drop('default',axis=1)
df_a = df_a.drop('housing',axis=1)
df_a = df_a.drop('loan',axis=1)
df_a = df_a.drop('pdays',axis=1)

Displaying Final Data to be interpreted

In [133]:
df_a.head(10)

,job,marital,education,contact,month,poutcome,y,balanceSummary,ageBand,defaultValue,housingVal,loanVal
0,unemployed,married,primary,cellular,oct,unknown,no,positive,ageBand3,defaultNo,houseNo,loanNo
1,services,married,secondary,cellular,may,failure,no,positive,ageBand3,defaultNo,houseYes,loanYes
2,management,single,tertiary,cellular,apr,failure,no,positive,ageBand3,defaultNo,houseYes,loanNo
3,management,married,tertiary,unknown,jun,unknown,no,positive,ageBand3,defaultNo,houseYes,loanYes
4,blue-collar,married,secondary,unknown,may,unknown,no,positive,ageBand5,defaultNo,houseYes,loanNo
5,management,single,tertiary,cellular,feb,failure,no,positive,ageBand3,defaultNo,houseNo,loanNo
6,self-employed,married,tertiary,cellular,may,other,no,positive,ageBand3,defaultNo,houseYes,loanNo
7,technician,married,secondary,cellular,may,unknown,no,positive,ageBand3,defaultNo,houseYes,loanNo
8,entrepreneur,married,tertiary,unknown,may,unknown,no,positive,ageBand4,defaultNo,houseYes,loanNo
9,services,married,primary,cellular,apr,failure,no,negative,ageBand4,defaultNo,houseYes,loanYes


Displaying all Unique Values

In [139]:
for col in df_a:
    print(col, ":", df_a[col].unique())

job : ['unemployed' 'services' 'management' 'blue-collar' 'self-employed'
 'technician' 'entrepreneur' 'admin.' 'student' 'housemaid' 'retired'
 'unknown']
marital : ['married' 'single' 'divorced']
education : ['primary' 'secondary' 'tertiary' 'unknown']
contact : ['cellular' 'unknown' 'telephone']
month : ['oct' 'may' 'apr' 'jun' 'feb' 'aug' 'jan' 'jul' 'nov' 'sep' 'mar' 'dec']
poutcome : ['unknown' 'failure' 'other' 'success']
y : ['no' 'yes']
balanceSummary : ['positive' 'negative' 'veryPositive' 'veryNegative']
ageBand : ['ageBand3' 'ageBand5' 'ageBand4' 'ageBand1' 'ageBand2']
defaultValue : ['defaultNo' 'defaultYes']
housingVal : ['houseNo' 'houseYes']
loanVal : ['loanNo' 'loanYes']


# Part 2
Changing all non-numeric into category data for Part 3

In [134]:
df_b['job'] = df_b['job'].astype('category')
df_b['job'] = df_b['job'].cat.codes
df_b['marital'] = df_b['marital'].astype('category')
df_b['marital'] = df_b['marital'].cat.codes
df_b['education'] = df_b['education'].astype('category')
df_b['education'] = df_b['education'].cat.codes
df_b['default'] = df_b['default'].astype('category')
df_b['default'] = df_b['default'].cat.codes
df_b['contact'] = df_b['contact'].astype('category')
df_b['contact'] = df_b['contact'].cat.codes
df_b['month'] = df_b['month'].astype('category')
df_b['month'] = df_b['month'].cat.codes
df_b['poutcome'] = df_b['poutcome'].astype('category')
df_b['poutcome'] = df_b['poutcome'].cat.codes
df_b['housing'] = df_b['housing'].astype('category')
df_b['housing'] = df_b['housing'].cat.codes
df_b['loan'] = df_b['loan'].astype('category')
df_b['loan'] = df_b['loan'].cat.codes
df_b['y'] = df_b['y'].astype('category')
df_b['y'] = df_b['y'].cat.codes
df_b = df_b.drop('pdays',axis=1)
df_b['balance'] = (df_b['balance'] - df_b['balance'].min()) / ( df_b['balance'].max() - df_b['balance'].min())
print("Post Conversion of Data into numeric, dropping pdays, and normalizing balance:")
df_b.head(10)

Post Conversion of Data into numeric, dropping pdays, and normalizing balance:


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,previous,poutcome,y
0,30,10,1,0,0,0.068455,0,0,0,19,10,79,1,0,3,0
1,33,7,1,1,0,0.108750,1,1,0,11,8,220,1,4,0,0
2,35,4,2,2,0,0.062590,1,0,0,16,0,185,1,1,0,0
3,30,4,1,2,0,0.064281,1,1,2,3,6,199,4,0,3,0
4,59,1,1,1,0,0.044469,1,0,2,5,8,226,1,0,3,0
5,35,4,2,2,0,0.054496,0,0,0,23,3,141,2,3,0,0
6,36,6,1,2,0,0.048590,1,0,0,14,8,341,1,2,1,0
7,39,9,1,1,0,0.046442,1,0,0,6,8,151,2,0,3,0
8,41,2,1,2,0,0.047436,1,0,2,14,8,57,2,0,3,0
9,43,7,1,0,0,0.043288,1,1,0,17,0,313,1,2,0,0


Creating X and Y for machine learning Algorithms

In [135]:
Y = df_b['y']
df_b = df_b.drop('y',axis=1)
X = df_b

Partitioning data into training and testing sets

In [136]:
x_train, x_test, y_train, y_test  = train_test_split( X, Y,  test_size=0.20, random_state=1)
print('len(x_train), len(y_train) = ' + str(len(x_train)) + ',' + str(len(y_train)) )
print('len(x_test), len(y_test) = ' + str(len(x_test)) + ',' + str(len(y_test)) )

len(x_train), len(y_train) = 3616,3616
len(x_test), len(y_test) = 905,905


Running a decision Tree

In [137]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train,y_train)
print(clf)
y_predicted = clf.predict(x_test)
print('\nconfusion_matrix from decision tree:')
print(confusion_matrix( y_test, y_predicted ) )
accuracy = accuracy_score(y_test, y_predicted)
recall = recall_score(y_test, y_predicted)
precision = precision_score(y_test, y_predicted)
print('accuracy = ' + str(accuracy))
print('recall = ' + str(recall))
print('precision = ' + str(precision))

DecisionTreeClassifier()

confusion_matrix from decision tree:
[[726  64]
 [ 68  47]]
accuracy = 0.8541436464088398
recall = 0.40869565217391307
precision = 0.42342342342342343


Running GaussianNB

In [138]:
model = GaussianNB()
model.fit(x_train,y_train)
gausianNB_predicted = model.predict(x_test)
print('\nconfusion_matrix from Gaussian naive bayes:')
print(confusion_matrix( y_test, gausianNB_predicted ) )
accuracy = accuracy_score(y_test, gausianNB_predicted)
recall = recall_score(y_test, gausianNB_predicted )
precision = precision_score(y_test, gausianNB_predicted)
print('accuracy = ' + str(accuracy))
print('recall = ' + str(recall))
print('precision = ' + str(precision))


confusion_matrix from Gaussian naive bayes:
[[731  59]
 [ 65  50]]
accuracy = 0.8629834254143647
recall = 0.43478260869565216
precision = 0.45871559633027525
